# Importação de bibliotecas e leitura dos dados

Inicialmente vamos importar as bibliotecas que serão utilizadas para a resolução do problema.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

Agora podemos realizar a leitura dos conjuntos de dados de treino e teste (já previamente separados neste caso).

In [2]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

Apenas para verificar se a leitura foi executada corretamente, vamos dar uma olhada nas 5 primeiras entradas do dataframe dos dados de treinamento.

In [3]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Podemos ver que a leitura foi executada com sucesso.<br>
Agora vamos analisar o tamanho dos conjuntos de dados.

In [4]:
print(train_data.shape)
print(test_data.shape)

(891, 12)
(418, 11)


O conjunto de treinamento parece ter um tamanho razoável para gerar um modelo de classificação.<br>
Com isso, podemos passar agora para a análise dos atributos.

# Análise dos atributos

Inicialmente vamos dar uma olhada em quais são os atributos do dataset, juntamente com uma breve descrição de cada um e os valores que podem assumir.

In [5]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

<table style="border: 1px solid black">
    <tr style="border: 1px solid black">
        <th style="border: 1px solid black; text-align: left">Variável</th>        
        <th style="border: 1px solid black; text-align: left">Descrição</th>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">PassengerId</td>        
        <td style="border: 1px solid black; text-align: left">Identificador do passageiro</td>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">Survived</td>        
        <td style="border: 1px solid black; text-align: left">Indica se o passageiro sobreviveu (1) ou não (0) ao desastre</td>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">Pclass</td>
        <td style="border: 1px solid black; text-align: left">Classe na qual o passageiro estava viajando, sendo 1 para primeira; 2 para segunda; e 3 para terceira</td>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">Name</td>
        <td style="border: 1px solid black; text-align: left">Nome do passageiro</td>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">Sex</td>
        <td style="border: 1px solid black; text-align: left">Sexo do passageiro</td>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">Age</td>
        <td style="border: 1px solid black; text-align: left">Idade do passageiro</td>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">SibSp</td>
        <td style="border: 1px solid black; text-align: left">Quantidade de irmãos e cônjuges a bordo</td>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">Parch</td>
        <td style="border: 1px solid black; text-align: left">Quantidade de pais e filhos do passageiro a bordo</td>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">Ticket</td>
        <td style="border: 1px solid black; text-align: left">Número da passagem</td>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">Fare</td>
        <td style="border: 1px solid black; text-align: left">Preço pago pela passagem</td>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">Cabin</td>
        <td style="border: 1px solid black; text-align: left">Número da cabine onde o passageiro estava viajando</td>
    </tr>
    <tr style="border: 1px solid black">
        <td style="border: 1px solid black; text-align: left">Embarked</td>
        <td style="border: 1px solid black; text-align: left">Porto no qual o passageiro embarcou: C para Cherbourg; Q para Queenstown; e S para Southampton</td>
    </tr>
</table>

# Limpeza de atributos irrelevantes

Como podemos ver, alguns atributos são claramente irrelevantes para a tarefa de classificação. Portanto, o ideal é que sejam removidos do dataset pois, caso contrário, podem atrapalhar a performance do algoritmo.

Os atributos citados são:
<ul>
    <li><b>name</b>: claramente o nome do passageiro não deve influenciar nas suas chances de sobrevivência</li> 
    <li><b>ticket</b>: o número do bilhete não traz nenhuma informação relevante sobre o passageiro, inclusive pode até ser aleatório</li>
    <li><b>cabin</b>: a cabine teoricamente poderia trazer uma informação sobre onde o passageiro poderia estar na hora do acidente, porém esta informação já está de certa forma embutida no atributo <b>Pclass</b></li>
</ul>

Desta forma, vamos retirar os atributos mencionados dos conjuntos de treino e de teste.

In [6]:
train_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
test_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

Vamos ver se a operação foi realizada com sucesso.

In [7]:
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


Tudo certo! Podemos agora seguir para a próxima etapa da preparação dos dados.

# Verificação de dados faltantes

Com um comando do framework pandas podemos analisar quais atributos possuem valores faltantes e quantos são.<br>
Esta análise permite tomar uma decisão mais acertada sobre o que fazer em relação a esses dados.

In [8]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         2
dtype: int64

In [9]:
train_data[train_data['Survived'] == 1]['Age'].mean()

28.343689655172415

In [10]:
test_data.isnull().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
SibSp           0
Parch           0
Fare            1
Embarked        0
dtype: int64

Como podemos ver, os atributos com valores faltantes são <b>Age</b>, <b>Embarked</b> e <b>Fare</b>.

O atributo <b>Age</b> merece uma atenção especial pois possui uma quantidade considerável de valores faltantes e é muito importante para a classificação, haja vista que o procedimento padrão em situação de evacuação é priorizar mulheres e crianças.

Neste caso, vamos adotar uma estratégia bastante simples inicialmente, que é imputar os valores faltantes apenas com a média das idades dos passageiros.

In [11]:
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(), inplace=True)

Agora precisamos decidir o que fazer em relação ao atributo <b>Embarked</b>.

Diferentemente do atributo <b>Age</b>, este atributo teoricamente deve ter uma influência menor na classificação e além disso o número de valores faltantes é muito baixo, apenas 2.

Sendo assim, uma estratégia interessante para preencher esses valores seria simplesmente colocar um valor aleatoriamente escolhido entre os possíveis 'C', 'Q' e 'S'.

In [12]:
fill_list = ['C', 'Q', 'S']
train_data['Embarked'].fillna(pd.Series(np.random.choice(fill_list, size=len(train_data.index))), inplace=True)

Por fim, precisamos decidir o que fazer com o atributo <b>Fare</b>.

Semelhante ao <b>Embarked</b>, este atributo não deve ter tanta influência na classificação e a informação que ele traz já está de certa forma contida na classe em que o passageiro viajava. E mais uma vez o número de valores faltantes é bem pequeno, apenas 1 no conjunto de teste.

Porém, não é possível escolher o valor aleatoriamente de uma lista como foi feito com <b>Embarked</b>. Portanto, vamos adotar a novamente de imputar o valor faltante com a média dos outros valores do conjunto de teste.

In [13]:
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace=True)

# Encoding dos atributos categóricos

Como próximo passo, vamos fazer o encoding dos atributos categóricos para transformá-los em dados numéricos. Esta transformação é necessária para deixar o conjunto de dados em formato aceitável para todos os algoritmos de classificação que serão utilizados nesta análise.

Primeiramente vamos relembrar quais são os atributos categóricos que queremos transformar.

In [14]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Sex             object
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked        object
dtype: object

Portanto, os atributos que queremos fazer o encoding são <b>Sex</b> e <b>Embarked</b>.

Como o número de dimensões do nosso dataset é bem baixo, a melhor estratégia sem dúvida é usar o One Hot Encoding. Esta técnica divide o atributo categórico em novas colunas (uma para cada um dos possíveis valores) e preenche os valores com 0 ou 1.

Vejamos como fica a aplicação do One Hot Encoding ao nosso dataset.

In [15]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)

In [16]:
train_data.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1


Como podemos ver, o atributo <b>Sex</b> foi dividido em <b>Sex_female</b> e <b>Sex_male</b>; enquanto que o atributo <b>Embarked</b> foi dividido em <b>Embarked_C</b>, <b>Embarked_Q</b> e <b>Embarked_S</b>.

Finalmente nosso dataset está pronto para a aplicação dos algoritmos de classificação.

# Algoritmos de classificação

Antes de aplicarmos os algoritmos, precisamos ressaltar alguns pontos:
<ul>
    <li>Em todos os algoritmos será aplicada a técnica de Cross-Validation para obter o score final</li>
    <li>O score será calculado sobre o conjunto train_data tratado anteriormente</li>
    <li>O algoritmo que obtiver o maior score no conjunto train_data será aplicado ao conjunto test_data para obter a solução final a ser submetida ao Kaggle</li>
    <li>Em todos os algoritmos será feito um Grid Search para obter os melhores hiperparâmetros</li>
</ul>

In [17]:
#train_data_X = train_data.loc[:, train_data.columns != 'Survived' and train_data.columns !='PassengerId'].values
train_data_X = train_data.drop(['Survived', 'PassengerId'], axis=1).values
#train_data_y = train_data.loc[:, train_data.columns == 'Survived'].values
train_data_y = train_data['Survived'].values

## Algoritmo #1: Árvores de decisão

In [18]:
tree_parameters = {'max_depth':[3,5,7,10]}

acc_tree = 0

for i in range(0,5):
    X_train, X_test, y_train, y_test = train_test_split(train_data_X, train_data_y, test_size=0.4)    

    grid_tree = GridSearchCV(DecisionTreeClassifier(), tree_parameters, cv=3)
    grid_tree.fit(X_train, y_train.ravel())

    tree = DecisionTreeClassifier(max_depth=grid_tree.best_params_['max_depth'])
    tree.fit(X_train, y_train.ravel())

    acc_tree = acc_tree + tree.score(X_test, y_test)

acc_tree = acc_tree / 5

print("A acurácia média da Árvore de Decisão é: %0.2f" % (acc_tree))

A acurácia média da Árvore de Decisão é: 0.81


## Algoritmo #2: KNN (K-vizinhos mais próximos)

In [19]:
knn_parameters = {'n_neighbors':[1,5,11,15,21,25]}

acc_knn = 0

for i in range(0,5):
    X_train, X_test, y_train, y_test = train_test_split(train_data_X, train_data_y, test_size=0.4)    

    grid_knn = GridSearchCV(KNeighborsClassifier(), knn_parameters, cv=3)
    grid_knn.fit(X_train, y_train.ravel())

    knn = KNeighborsClassifier(n_neighbors=grid_knn.best_params_['n_neighbors'])
    knn.fit(X_train, y_train.ravel())

    acc_knn = acc_knn + knn.score(X_test, y_test)

acc_knn = acc_knn / 5

print("A acurácia média do KNN é: %0.2f" % (acc_knn))

A acurácia média do KNN é: 0.69


## Algoritmo #3: SVM (Support Vector Machine)

In [20]:
svm_parameters = {'C':[2**(-5), 2**(0), 2**(5), 2**(10)],
 'gamma':[2**(-15), 2**(-10), 2**(-5), 2**(0), 2**(5)]}

acc_svm = 0

for i in range(0,5):
    X_train, X_test, y_train, y_test = train_test_split(train_data_X, train_data_y, test_size=0.4)    

    grid_svm = GridSearchCV(SVC(kernel='rbf'), svm_parameters, cv=3)
    grid_svm.fit(X_train, y_train.ravel())

    svm = SVC(C=grid_svm.best_params_['C'], gamma=grid_svm.best_params_['gamma'], kernel='rbf')
    svm.fit(X_train, y_train.ravel())

    acc_svm = acc_svm + svm.score(X_test, y_test)

acc_svm = acc_svm / 5

print("A acurácia média do SVM é: %0.2f" % (acc_svm))

A acurácia média do SVM é: 0.79


## Algoritmo #4: Redes Neurais

In [21]:
nn_parameters = {'hidden_layer_sizes':[5,10,20]}

acc_nn = 0

for i in range(0,5):
    X_train, X_test, y_train, y_test = train_test_split(train_data_X, train_data_y, test_size=0.4)    

    grid_nn = GridSearchCV(MLPClassifier(solver='lbfgs'), nn_parameters, cv=3)
    grid_nn.fit(X_train, y_train.ravel())
    
    nnet = MLPClassifier(hidden_layer_sizes=grid_nn.best_params_['hidden_layer_sizes'], solver='lbfgs')
    nnet.fit(X_train, y_train.ravel())

    acc_nn = acc_nn + nnet.score(X_test, y_test)

acc_nn = acc_nn / 5

print("A acurácia média da Rede Neural é: %0.2f" % (acc_nn))

A acurácia média da Rede Neural é: 0.81


## Algoritmo #5: Naive Bayes

In [22]:
acc_nb = 0

for i in range(0,5):
    X_train, X_test, y_train, y_test = train_test_split(train_data_X, train_data_y, test_size=0.4)    

    #grid_nn = GridSearchCVGaussian, nn_parameters, cv=3)
    #grid_nn.fit(X_train, y_train.ravel())
    
    nb = GaussianNB()
    nb.fit(X_train, y_train.ravel())

    acc_nb = acc_nb + nb.score(X_test, y_test)

acc_nb = acc_nb / 5

print("A acurácia média do Naive Bayes é: %0.2f" % (acc_nb))

A acurácia média do Naive Bayes é: 0.78


## Algoritmo #6: Random Forest

In [23]:
rf_parameters = {'max_features':[2,5,9,10],'n_estimators':[10,50,100,200]}

acc_rf = 0

for i in range(0,5):
    X_train, X_test, y_train, y_test = train_test_split(train_data_X, train_data_y, test_size=0.4)    

    grid_rf = GridSearchCV(RandomForestClassifier(), rf_parameters, cv=3)
    grid_rf.fit(X_train, y_train.ravel())
        
    rf = RandomForestClassifier(max_features=grid_rf.best_params_['max_features'], n_estimators=grid_rf.best_params_['n_estimators'])
    rf.fit(X_train, y_train.ravel())

    acc_rf = acc_rf + rf.score(X_test, y_test)

acc_rf = acc_rf / 5

print("A acurácia média do Random Forest é: %0.2f" % (acc_rf))

A acurácia média do Random Forest é: 0.80


## Algoritmo #7: GBM (Gradient Boosting Machine)

In [24]:
gbm_parameters = {'n_estimators':[5,10,30,70],'learning_rate':[0.1,0.05],'max_depth':[3,5,7]}

acc_gbm = 0

for i in range(0,5):
    X_train, X_test, y_train, y_test = train_test_split(train_data_X, train_data_y, test_size=0.4)    

    grid_gbm = GridSearchCV(GradientBoostingClassifier(), gbm_parameters, cv=3)
    grid_gbm.fit(X_train, y_train.ravel())
            
    gbm = GradientBoostingClassifier(n_estimators=grid_gbm.best_params_['n_estimators'],
     learning_rate=grid_gbm.best_params_['learning_rate'], max_depth=grid_gbm.best_params_['max_depth'])
    gbm.fit(X_train, y_train.ravel())

    acc_gbm = acc_gbm + gbm.score(X_test, y_test)

acc_gbm = acc_gbm / 5

print("A acurácia média do Gradient Boosting Machine é: %0.2f" % (acc_gbm))

A acurácia média do Gradient Boosting Machine é: 0.81


## Algoritmo #8: AdaBoost

In [25]:
ada_parameters = {'n_estimators':[5,10,30,70],'learning_rate':[0.1,0.05]}

acc_ada = 0

for i in range(0,5):
    X_train, X_test, y_train, y_test = train_test_split(train_data_X, train_data_y, test_size=0.4)    

    grid_ada = GridSearchCV(AdaBoostClassifier(), ada_parameters, cv=3)
    grid_ada.fit(X_train, y_train.ravel())
            
    ada = AdaBoostClassifier(n_estimators=grid_ada.best_params_['n_estimators'],
     learning_rate=grid_ada.best_params_['learning_rate'])
    ada.fit(X_train, y_train.ravel())

    acc_ada = acc_ada + ada.score(X_test, y_test)

acc_ada = acc_ada / 5

print("A acurácia média do AdaBoost é: %0.2f" % (acc_ada))

A acurácia média do AdaBoost é: 0.82


# Aplicação de uma técnica de ensemble

Agora vamos aplicar uma técnica de ensemble conhecida como Soft Majority Voting, em que combinaremos 5 classificadores  com pesos definidos de acordo com as acurácias previamente obtidas.

In [26]:
acc_vote = 0

for i in range(0,5):
    X_train, X_test, y_train, y_test = train_test_split(train_data_X, train_data_y, test_size=0.4) 
    
    tree = DecisionTreeClassifier(max_depth=grid_tree.best_params_['max_depth'])
    tree.fit(X_train, y_train.ravel())
    
    knn = KNeighborsClassifier(n_neighbors=grid_knn.best_params_['n_neighbors'])
    knn.fit(X_train, y_train.ravel())

    svm = SVC(C=grid_svm.best_params_['C'], gamma=grid_svm.best_params_['gamma'], kernel='rbf', probability=True)
    svm.fit(X_train, y_train.ravel())
    
    nnet = MLPClassifier(hidden_layer_sizes=grid_nn.best_params_['hidden_layer_sizes'], solver='lbfgs')
    nnet.fit(X_train, y_train.ravel())
    
    nb = GaussianNB()
    nb.fit(X_train, y_train.ravel())
    
    vote = VotingClassifier(estimators=[('dt', tree), ('knn', knn), ('svm', svm), ('nnet', nnet), ('nb', nb)], voting='soft', weights=[3,1,2,2,2])    
    vote.fit(X_train, y_train.ravel())
    
    acc_vote = acc_vote + vote.score(X_test, y_test)

acc_vote = acc_vote / 5

print("A acurácia média do ensemble Soft Majority Voting é: %0.2f" % (acc_vote))

A acurácia média do ensemble Soft Majority Voting é: 0.82


# Preparação do arquivo de submissão para o Kaggle

Como vimos, a Árvore de Decisão foi o algoritmo com o melhor desempenho em uma validação cruzada, portanto será o algoritmo utilizado para a previsão final, com os melhores hiperparâmetros definidos pelo GridSearch.

Primeiramente, vamos fazer um novo treinamento do algoritmo mas agora utilizando todo o conjunto de treinamento.

In [27]:
#gbm = GradientBoostingClassifier(n_estimators=grid_gbm.best_params_['n_estimators'],
#     learning_rate=grid_gbm.best_params_['learning_rate'], max_depth=grid_gbm.best_params_['max_depth'])
#gbm.fit(train_data_X, train_data_y.ravel())

tree = DecisionTreeClassifier(max_depth=grid_tree.best_params_['max_depth'])
tree.fit(train_data_X, train_data_y.ravel())

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Agora vamos criar um novo dataframe do Pandas no formato em que os dados devem ser submetidos ao Kaggle.

In [28]:
submission = pd.DataFrame()
submission['PassengerId'] = test_data['PassengerId']
submission['Survived'] = gbm.predict(test_data.drop(['PassengerId'], axis=1).values)

Por fim, basta escrever o resultado em um arquivo csv que já estará pronto para a submissão.

In [29]:
submission.to_csv('submission.csv', index=False)